# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298367494556                   -0.85    4.9
  2   -8.300200190532       -2.74       -1.25    1.0
  3   -8.300448883940       -3.60       -1.87    3.6
  4   -8.300461761920       -4.89       -2.72    1.1
  5   -8.300463852143       -5.68       -3.07    8.4
┌ Warning: Eigensolver not converged
│   iterations =
│    8-element Vector{Int64}:
│       3
│       2
│       2
│       1
│       2
│       1
│       1
│     100
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:36
  6   -8.300464223203       -6.43       -3.22   14.0
  7   -8.300464420806       -6.70       -3.35    6.4
  8   -8.300464535190       -6.94       -3.49    1.9
  9   -8.300464628528       -7.03       -3.75    1.2
 10   -8.300464636580       -8.09       -3.90    1.6
 11   -8.300464642443       -8.23       -4.22    1.0


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67457301051                   -0.70    6.1
  2   -16.67867880434       -2.39       -1.14    1.5
  3   -16.67922805802       -3.26       -1.86    4.4
  4   -16.67927787694       -4.30       -2.75    2.1
  5   -16.67928566323       -5.11       -3.19    4.4
  6   -16.67928620857       -6.26       -3.49    3.6
  7   -16.67928622119       -7.90       -3.94    1.0
  8   -16.67928622391       -8.57       -4.59    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32481625226                   -0.56    6.6
  2   -33.33410028796       -2.03       -0.99    1.1
  3   -33.33602329120       -2.72       -1.71    3.8
  4   -33.33618490442       -3.79       -2.64    2.6
  5   -33.33692732923       -3.13       -2.68    5.5
  6   -33.33693842727       -4.95       -2.93   12.2
  7   -33.33694389142       -5.26       -4.05    2.1


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298526222259                   -0.85    4.6
  2   -8.300276648050       -2.76       -1.59    1.0
  3   -8.300421453468       -3.84       -2.38    6.4
┌ Warning: Eigensolver not converged
│   iterations =
│    8-element Vector{Int64}:
│      41
│       3
│       3
│       3
│       3
│       3
│       3
│     100
└ @ DFTK ~/work/DFTK.jl/DFTK.jl/src/scf/self_consistent_field.jl:36
  4   -8.300318320362   +   -3.99       -2.17   19.9
  5   -8.300464521667       -3.84       -3.72    2.9
  6   -8.300464606565       -7.07       -4.02    4.2


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32597784721                   -0.56    6.4
  2   -33.28117620523   +   -1.35       -1.25    1.1
  3   +10.89372851393   +    1.65       -0.24    8.6
  4   -33.31229743811        1.65       -1.60    7.6
  5   -32.91538907015   +   -0.40       -1.18    5.2
  6   -30.57077975138   +    0.37       -0.84    5.2
  7   -33.31744237311        0.44       -1.85    7.4
  8   -33.32375648976       -2.20       -1.85    2.2
  9   -33.33469270308       -1.96       -2.09    2.4
 10   -33.33661675908       -2.72       -2.69    2.4
 11   -33.33684579218       -3.64       -2.90    2.9
 12   -33.33685908510       -4.88       -2.98    1.4
 13   -33.33693599783       -4.11       -3.27    3.0
 14   -33.33694293339       -5.16       -3.70    3.0
 15   -33.33694200484   +   -6.03       -3.85    3.2
 16   -33.33694312988       -5.95       -4.05    3.4


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.